In [8]:
import pandas as pd

df = pd.read_excel('webscraping/data_unpacked.xlsx')
print(df.head())

                                                 url  \
0  https://lh3.googleusercontent.com/ci/ACqFvF1fa...   
1  https://lh3.googleusercontent.com/IN4tL8PgTXxm...   
2  https://lh3.googleusercontent.com/b6TOe8s5BtvR...   
3  https://lh3.googleusercontent.com/ci/ACqFvF0Lo...   
4  https://lh3.googleusercontent.com/LRWEui8Dl47I...   

                        title                            creator  \
0   Girl with a Pearl Earring                   Johannes Vermeer   
1             American Gothic   Grant Wood (American, 1891-1942)   
2       Luncheon on the Grass                                NaN   
3         Lady with an Ermine                  Leonardo da Vinci   
4           John Quincy Adams                                NaN   

                             created                        data.Artist  \
0   c. 1665  (digitized by Madpixel)                             artist   
1                               1930   Grant Wood (American, 1891-1942)   
2                            

In [11]:
'first' or 'second'
False or 'second'
F or 'second'

'second'